In [1]:
# Heterogeneous spreading

In [9]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
#from joblib import Parallel, delayed
#import multiprocessing

def first_infection(G,list_infected=[],list_recovered=[]):
    """
    Apply the first infection on a given graph.

    Args:
        G (Graph): 
        list_infected (list): List of first infected nodes
        list_recovered (list): List of first recovered nodes

    Returns:
        Graph: Graph corresponding to the initial state of the epidemic process

    """
    
    # Set the state of nodes to 'S' and the state of edges to 'SS'
    nx.set_node_attributes(G,'S','state')
    nx.set_edge_attributes(G,'SS','state')
    
    # Infect randomly a node if none are given, or infect the one given
    if len(list_infected)==0:
        N=G.number_of_nodes()
        N0=random.randint(0,N-1)
        G.nodes[N0]['state']='I'
        list_infected=[N0]
    else:
        for node in list_infected:
            G.nodes[node]['state']='I'

    # Infect the neighbor edge of the infected nodes
    for node1 in list_infected:
        for node2 in list(G.neighbors(node1)):
            n1,n2=sorted([node1,node2])
            G[n1][n2]['state']=G.nodes[n1]['state']+G.nodes[n2]['state']

    # If a list of recovered nodes is given, add the information    
    for node1 in list_recovered:
        G.nodes[node1]['state']='R'

        for node2 in list(G.neighbors(node1)):
            n1,n2=sorted([node1,node2])
            G[n1][n2]['state']=G.nodes[n1]['state']+G.nodes[n2]['state']
          
    return G


def create_watts_strogatz_graph(n,k,p,seed,beta,gamma,list_infected=[],list_recovered=[],list_heterogeneous_nodes=[],
                 list_heterogeneous_edges=[],heterogeneous_nodes_weight=[],heterogeneous_edges_weight=[]):
    
    """
    Create a Watts-Strogatz graph and apply the first infection on it.

    Args:
        n (int): Number of nodes of the Watts-Strogatz graph
        k (int): Number of neighbors (nearest) of the Watts-Strogatz graph
        p (float): Probability to rewire a link of the Watts-Strogatz graph
        seed (int): Random seed of the Watts-Strogatz graph
        beta (float): Infection rate 
        gamma (float): Recovery rate
        list_infected (list): List of first infected nodes
        list_recovered (list): List of first recovered nodes
        list_heterogeneous_nodes (list): List of heterogeneous nodes (with a different gamma)
        list_heterogeneous_edges (list): List of heterogeneous edges (with a different beta)
        heterogeneous_nodes_weight (list): Gamma values associated with the heterogeneous nodes
        heterogeneous_edges_weight (list): Beta values associated with the heterogeneous edges


    Returns:
        Graph: Graph corresponding to the initial state of the epidemic process whom 
        the beta and gamma values have been applied

    """
    
    # Create the graph
    G=nx.watts_strogatz_graph(n,k,p,seed=seed)
    
    # Infect the first nodes and recover nodes if some are given 
    G = first_infection(G,list_infected,list_recovered)
    
    # Apply the beta and gamma values to the edges and nodes
    nx.set_edge_attributes(G,beta,'weight')
    nx.set_node_attributes(G,gamma,'weight')
    for node in range(len(list_heterogeneous_nodes)):
        G.nodes[list_heterogeneous_nodes[node]]['weight']=heterogeneous_nodes_weight[node]
    for node in range(len(list_heterogeneous_edges)):
        G[list_heterogeneous_edges[node][0]][list_heterogeneous_edges[node][1]]['weight']=heterogeneous_edges_weight[node]

    return G




In [10]:

n =100 
k = 4 
p = 0.5 
seed =10 
beta = 0.5
gamma = 0.5
G_WS = create_watts_strogatz_graph(n,k,p, seed, beta,gamma)